In [ ]:
# author: Tom Stone <tomstone@stanford.edu>
# author: Proloy Das <email:proloyd94@gmail.com>
# License: BSD (3-clause)
%matplotlib widget
%matplotlib widget
import os
import numpy as np
from scipy import signal, fft
from pathlib import Path

from matplotlib import pyplot
from lab1_utils import *    # Familiar line? Now you are statring to appereciate this, right? :)


pyplot.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "figure.constrained_layout.use": True,
    "savefig.dpi": 300
})

notebook: Knock, knock!

you: Who is there?

notebook: Your familar line of code.

In [ ]:
rng = np.random.default_rng(2345)

## Generating an AR(4) signal

For this example, we begin with generating 1024 samples of the following AR(4) process:
$$\begin{aligned}
x_k = 3.28285739 x_{k-1} - 4.61269743 x_{k-2} + 3.21388527 x_{k-3} - 0.95865639 x_{k-4} + w_k; \ \ w_k \sim \mathcal{N}(0, 1)
\end{aligned}$$
We will start with $x_0=0, x_1=0, x_2=0$, and $x_3=0$, generate 1024 + 50 samples, and discard first 50 samples.
Also we will create the time indices corresponding the white noise sequence.

In [ ]:
phi = np.array([1., -3.28285739, 4.61269743, -3.21388527, 0.95865639])  
# Note that we need to reverse the sign of AR coeffiences, when they are given in this form.
n = 1024
# Generate the samples of driving noise.
w = rng.normal(size=n+50)
# Generate the AR process samples starting from k = 4.
x = np.zeros(n + 50)
for i in range(4, n+50):
    x[i] = - phi[1] * x[i-1] - phi[2] * x[i-2] - phi[3] * x[i-3] - phi[4] * x[i-4] + w[i]
    # x[i] = - np.inner(phi[1:], x[i-phi.shape[-1]+1:i][::-1]) + w[i]
ar4 = x[50:]

# The time indices associated with the samples.
time_indices = np.arange(1024)

Plot the white noise againt time using matplotlib.

In [ ]:
fig1, ax = pyplot.subplots(figsize=(8, 2))
ax.plot(time_indices, ar4, linewidth=0.5)
ax.set_xlabel('$t$')
ax.set_ylabel('$ar2$')
ax.set_ylim([-120, 120])
_ = ax.set_title('AR(4) process')

Now we compute the autocovaraince sequence from the realization that we generated.

In [ ]:
# Sampe autocorrelation
max_lag = ar4.shape[-1] - 1
sample_acov, lags = compute_autocovaraince(ar4, max_lag)

The following lines of code generates the true autocovariance sequence from the AR(4) process definition.

In [ ]:
true_acov, lags_ = compute_theoritical_ar_acov(phi, max_lag)

Lets take a quick look at the autocovarainces to gauze the estimation error, i.e., the noise.

In [ ]:
fig1, ax = pyplot.subplots(figsize=(8, 2))
ax.plot(lags, sample_acov, linewidth=1, color='r', label='Sample')
ax.plot(lags_, true_acov, linewidth=1, color='b', label='True')
ax.set_ylim([-1000.0, 1600])
ax.legend()
ax.set_xlim([-1024, 1024])
_ = ax.set_title('Autocorrelation sequence plot')

Now we are ready for periodograms!

In [ ]:
S_xx_true = np.abs(fft.fft(true_acov))
freqs_ = np.linspace(0, 1, num=len(lags))

S_xx_est, freqs = compute_periodogram(ar4)


# Periodogram plot
fig1, ax = pyplot.subplots(figsize=(5, 2))
ax.plot(freqs, 10*np.log10(S_xx_est), linewidth=1, color='r', label='Sample')
ax.plot(freqs_, 10*np.log10(S_xx_true), linewidth=1, color='b', label='True')
ax.set_ylim([-30, 60])
ax.set_xlim([0., 0.5])
ax.legend()
_ = ax.set_title('Periodogram plot')

Bias!

How do we take care of that?
Tapering!

In [ ]:
win = signal.get_window('hann', 1024)

In [ ]:
times = np.arange(ar4.shape[-1])
fig, ax = pyplot.subplots(figsize=(8, 2))
ax.plot(times, ar4 / 100, linewidth=0.5)
ax.plot(times, win+2, linewidth=0.5, color='k')
ax.set_ylim([-1.2, 3.2])

In [ ]:
tapered_ar4 = ar4 * win
fig, ax = pyplot.subplots(figsize=(8, 2))
ax.plot(times, tapered_ar4 / 100, linewidth=0.5)
ax.set_ylim([-1.2, 1.2])

# Function number three, `periodogram_tapered`

In [ ]:
freq, S_yy_true = signal.freqz(1, phi, fs=1)

S_yy_est, freq_est = periodogram_tapered(ar4, win)

fig, ax = pyplot.subplots(figsize=(5, 2.5))
ax.plot(freq_est, 10*np.log10(S_yy_est), linewidth=0.5, label='Tapered estimate')
ax.plot(freq, 20*np.log10(np.abs(S_yy_true)), color='r', linewidth=0.5, label='True')
ax.set_ylabel("$10\log_{10}\widehat{S}(f)$ (dB)")
ax.set_xlabel("$fT$")
ax.set_title("AR(4) process")
ax.legend()
ax.set_xlim([0, 0.5])

## Question
What has the taper eliminated, the bias or the variance?

## Another taper
Let's look at the DPSS zero-sequence taper

In [ ]:
taper = signal.windows.dpss(1024, 4, Kmax=1)[0,:] * np.sqrt(1024) # need this normalizing constant for DPSS
fig, ax = pyplot.subplots(figsize=(6,3))
ax.plot(taper)

In [ ]:
freq, S_yy_true = signal.freqz(1, phi, fs=1)

S_yy_est, freq_est = periodogram_tapered(ar4, taper)

fig, ax = pyplot.subplots(figsize=(5, 2.5))
ax.plot(freq_est, 10*np.log10(S_yy_est), linewidth=0.5, label='Tapered estimate')
ax.plot(freq, 20*np.log10(np.abs(S_yy_true)), color='r', linewidth=0.5, label='True')
ax.set_ylabel("$10\log_{10}\widehat{S}(f)$ (dB)")
ax.set_xlabel("$fT$")
ax.set_title("AR(4) process")
ax.legend()
ax.set_xlim([0, 0.5])

## Function number four, `multitaper_periodogram`
There are more than one DPSS tapers! Write a function to average the results from multiple DPSS tapers.

Remember that you can control (1) the time-half-bandwidth product and (2) the number of tapers used

Implement the function `multitaper_periodogram`

In [ ]:
nw = 4
ntapers = 4

freq, S_yy_true = signal.freqz(1, phi, fs=1)

S_yy_est, freq_est = multitaper_periodogram(ar4, nw, ntapers)

fig, ax = pyplot.subplots(figsize=(5, 2.5))
ax.plot(freq_est, 10*np.log10(S_yy_est), linewidth=0.5, label='Multitaper estimate')
ax.plot(freq, 20*np.log10(np.abs(S_yy_true)), color='r', linewidth=0.5, label='True')
ax.set_ylabel("$10\log_{10}\widehat{S}(f)$ (dB)")
ax.set_xlabel("$fT$")
ax.set_title("AR(4) process")
ax.legend()
ax.set_xlim([0, 0.5])